In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(type('incremental_flag'))

## CREATING DIMENSION

## fetch relative columns

In [0]:
df_src = spark.sql('''select distinct (Branch_ID) as Branch_ID, BranchName
         from parquet.`abfss://silver@adlscardatalake.dfs.core.windows.net/carsales/`
         ''')

In [0]:
display(df_src)

##  dim Branch-sink Initial and incremental(Just bring the schema if table not exists)

In [0]:
if  spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
  df_sink = spark.sql('''select  dim_branch_key,Branch_ID, BranchName 
                    from  cars_catalog.gold.dim_branch
                    ''')
else:
  df_sink = spark.sql('''select 1 as dim_branch_key,Branch_ID,BranchName 
                    from  parquet.`abfss://silver@adlscardatalake.dfs.core.windows.net/carsales/` 
                    where 1= 0''')

In [0]:
df_filter = df_src.join(df_sink, df_src.Branch_ID == df_sink.Branch_ID, 'left').select(df_src.Branch_ID,df_src.BranchName,df_sink.dim_branch_key)

In [0]:
df_filter.display()

**df_filter_old**

In [0]:
df_filter_old = df_filter.filter(df_filter.dim_branch_key.isNotNull())

In [0]:
df_filter_old.display()

**df_filter_new**

In [0]:
df_filter_new = df_filter.filter(df_filter.dim_branch_key.isNull()).select(df_src['Branch_ID'],df_src['BranchName'])

In [0]:
df_filter_new.display()

## Create Surrogate key

**fetch the max surrogate key***

In [0]:
if (incremental_flag == '0') :
    max_value = 1
else:
   max_val_df = spark.sql("select max(dim_branch_key) from cars_catalog.gold.dim_branch")
   max_value = (max_val_df.collect()[0][0]) +1
print(max_value)

***create surrorgate key and add the max surrogate key***


In [0]:
df_filter_new = df_filter_new.withColumn('dim_branch_key', max_value + monotonically_increasing_id())

In [0]:
df_filter_new.display()

## create final_df = df_filter_old + df_filter_new

In [0]:
df_final = df_filter_old.union(df_filter_new)
df_final.display()


# SCD TYPE 1(UPSERT)

In [0]:
from delta.tables import DeltaTable

In [0]:
#source-df_Final,dest-dim_branch

    #Incremental run
src = df_final

if spark.catalog.tableExists('cars_catalog.gold.dim_branch'):
    delta_tbl = DeltaTable.forPath(spark,'abfss://gold@adlscardatalake.dfs.core.windows.net/dim_branch')    #creating DeltaTable object on top of dim_branch
    delta_tbl.alias('trg').merge (df_final.alias('src'), "trg.dim_branch_key == src.dim_branch_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
   #initial run
else:
    df_final.write.format('delta')\
        .mode('overwrite')\
        .option('path','abfss://gold@adlscardatalake.dfs.core.windows.net/dim_branch')\
        .saveAsTable('cars_catalog.gold.dim_branch')